In [ ]:
import pandas as pd 
import numpy as np

dm = pd.read_csv("6th Meet (Monday, March, 14th 2022)\Material\Data\motogpmandalika_clean2_1.csv", usecols=["label", "teksbersih"])
dm.columns = ["label", "teksbersih"]
dm.head()

In [ ]:
# convert list formated string to list
import ast

def convert_text_list(texts):
    texts = ast.literal_eval(texts)
    return [text for text in texts]

dm["tekslist"] = dm["teksbersih"].apply(convert_text_list)

print(dm["tekslist"][90])

print("\ntype : ", type(dm["tekslist"][90]))

In [ ]:
def calc_TF(document):
    #perhitungan jumlah kata
    TF_dict = {}
    for term in document:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    #perhitungan tf
    for term in TF_dict:
        TF_dict[term] = TF_dict[term] / len(document)
    return TF_dict

dm["TF_dict"] = dm['tekslist'].apply(calc_TF)

dm["TF_dict"].head()

In [ ]:
#hasil tf
index = 90

print('%20s' % "term", "\t", "TF\n")
for key in dm["TF_dict"][index]:
    print('%20s' % key, "\t", dm["TF_dict"][index][key])

In [ ]:
def calc_DF(tfDict):
    count_DF = {}
    for document in tfDict:
        for term in document:
            if term in count_DF:
                count_DF[term] += 1
            else:
                count_DF[term] = 1
    return count_DF

DF = calc_DF(dm["TF_dict"])


In [ ]:
#menghitung idf

n_document = len(dm)
def calc_IDF(__n_document, __DF):
    IDF_Dict = {}
    for term in __DF:
        IDF_Dict[term] = np.log(__n_document / (__DF[term] + 1))
    return IDF_Dict
  
#penyimpanan kamus idf
IDF = calc_IDF(n_document, DF)

In [ ]:
#perhitungan TF-IDF
def calc_TF_IDF(TF):
    TF_IDF_Dict = {}
    for key in TF:
        TF_IDF_Dict[key] = TF[key] * IDF[key]
    return TF_IDF_Dict

#penyimpanan variabel TF-IDF 
dm["TF-IDF_dict"] = dm["TF_dict"].apply(calc_TF_IDF)

In [ ]:
# memunculkan nilai TF-IDF
index = 90

print('%20s' % "term", "\t", '%10s' % "TF", "\t", '%20s' % "TF-IDF\n")
for key in dm["TF-IDF_dict"][index]:
    print('%20s' % key, "\t", dm["TF_dict"][index][key] ,"\t" , dm["TF-IDF_dict"][index][key])

In [ ]:
#matrik tf-idf
# pengurutan descending berdasarkan nilai DF  
sorted_DF = sorted(DF.items(), key=lambda kv: kv[1], reverse=True)[:50]

# pembuatan list kata dari pengurutan `sorted_DF`
unique_term = [item[0] for item in sorted_DF]

def calc_TF_IDF_Vec(__TF_IDF_Dict):
    TF_IDF_vector = [0.0] * len(unique_term)

    # For each unique word, if it is in the review, store its TF-IDF value.
    for i, term in enumerate(unique_term):
        if term in __TF_IDF_Dict:
            TF_IDF_vector[i] = __TF_IDF_Dict[term]
    return TF_IDF_vector

dm["TF_IDF_Vec"] = dm["TF-IDF_dict"].apply(calc_TF_IDF_Vec)

print("tampil baris pertama matrix TF_IDF_Vec Series\n")
print(dm["TF_IDF_Vec"][0])

print("\nukuran matrix : ", len(dm["TF_IDF_Vec"][0]))

In [ ]:
#menampilkan top 50 term tf-idf

# konversi ke dalam List
TF_IDF_Vec_List = np.array(dm["TF_IDF_Vec"].to_list())
 
sums = TF_IDF_Vec_List.sum(axis=0)

data = []

for col, term in enumerate(unique_term):
    data.append((term, sums[col]))
    
ranking = pd.DataFrame(data, columns=['term', 'rank'])
ranking.sort_values('rank', ascending=False)

In [ ]:
print('hello world')